In [14]:
#| default_exp music

In [15]:
#| export
from fasthtml.common import *
from fasthtml.components import Uk
from fh_frankenui.components import *
from fasthtml.components import Uk_icon
from fasthtml.svg import *
from fh_matplotlib import matplotlib2fasthtml
import numpy as np
from pathlib import Path
import matplotlib.pylab as plt

In [16]:
#| hide
from nbdev.showdoc import *

In [17]:
%%html
<script src="https://cdn.tailwindcss.com"></script>
<script src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit.min.js"></script>
<script src="https://cdn.jsdelivr.net/npm/uikit@3.21.6/dist/js/uikit-icons.min.js"></script>
<script type="module" src="https://unpkg.com/franken-wc@0.0.6/dist/js/wc.iife.js"></script>
<link rel="stylesheet" href="https://unpkg.com/franken-wc@0.0.6/dist/css/blue.min.css">
<style>
#notebook-container { max-width: none; }
.output_html * { list-style-type: none !important; }
.disabled {opacity: 0.5; pointer-events: none; cursor: default; text-decoration: none;}
</style>

In [18]:
from httpx import get
fh_fui = Path(os.path.expanduser('~/github/answerdotai/fh-frankenui/fh_frankenui'))
fh_fui_ex = Path(os.path.expanduser('~/github/answerdotai/fh-frankenui-examples/fh_frankenui_examples'))

fh_frankenui = (fh_fui / "components.py").read_text()

url = 'https://raw.githubusercontent.com/franken-ui/examples/master/src/pages/examples/music.astro'
music = get(url).text

url = 'https://raw.githubusercontent.com/franken-ui/examples/master/src/pages/examples/cards.astro'
cards = get(url).text

cards_in_python = (fh_fui_ex / 'cards.py').read_text()

url = 'https://raw.githubusercontent.com/AnswerDotAI/fasthtml/main/examples/adv_app.py'
idiomatic_fasthtml_app = get(url).text

In [19]:
%%ai 0 -c

I am implementing FrankenUI for FastHTML by reimplementing the FrankenUI examples from their docs in a python library.

I am doing this by building a library as I go, $`fh_frankenui`.  You should use this library as much as possible

Reference the $`idiomatic_fasthtml_app` to get a feel for fasthtml.

I previously implemented the $`cards` example, and the python implementation looked like $`cards_in_python`.

Now, I am looking to do the same with the $`music` example.

The goal is to develop the work-in-progress `fh_frankenui` library, so please use `fh_frankenui` as much as makes sense and recommend functions or improvements to `fh_frankenui` when it fits.

Please read all this information and confirm that you understand what I want to do by giving me a brief overview in your own words.

Before answering, let me know if any of the variables I supplied are invalid or empty.

I've reviewed the provided information and variables. All the variables appear to be valid and contain the expected content. Now, I'll provide a brief overview of your task as I understand it:

You're developing a Python library called `fh_frankenui` to implement FrankenUI components for FastHTML. Your approach is to recreate the FrankenUI examples from their documentation in Python, building up the library as you go. You've already implemented the 'cards' example, and now you're moving on to the 'music' example.

Your goals are:

1. Implement the 'music' example from FrankenUI in Python using FastHTML.
2. Utilize the existing `fh_frankenui` library as much as possible.
3. Suggest improvements or additional functions for `fh_frankenui` where appropriate.
4. Follow the idiomatic FastHTML style, referencing the `idiomatic_fasthtml_app` as a guide.

The process involves translating the HTML and JavaScript components from the FrankenUI examples into equivalent Python code using FastHTML's syntax and components. As you work through this, you're aiming to create reusable components and functions that can be added to the `fh_frankenui` library for future use.

Is this understanding correct? Would you like me to proceed with implementing the 'music' example or do you need any clarification?

In [137]:
from fastcore.all import tuplify

In [152]:
#| export
# What to do with this?
def SpacedTxtIcon(txt, icon, ratio, icon_right=True):
    #TODO: icon_right=False
    c = (P(txt),UkIcon(icon,ratio))
    if not icon_right: c = reversed(c)
    return Div(cls='uk-flex uk-flex-between uk-flex-middle uk-width-1-1')(*c)

def LAlignedTxtIcon(txt, icon, ratio, icon_right=True):
    # TODO: Test and make sure this works
    # TODO: How do you pass text with a color? Should txt be a string?  Or should you pass a P?  
    c = (P(txt),UkIcon(icon,ratio))
    if not icon_right: c = reversed(c)
    return Div(cls='space-x-6')(*c)

def SpacedPP(left, right=None):
    return Div(cls='uk-flex uk-flex-between uk-flex-middle uk-width-1-1')(P(left),P(right) if right else '')

def SpacedPPs(*c):
    return [SpacedPP(*tuplify(o)) for o in c]

def UkIcon(icon, ratio=1):
    return Span(uk_icon=f"icon: {icon}; ratio: {ratio}")

In [151]:
#| export

#TODO: Apply the above functions in here to finish that
# .disabled {opacity: 0.5; pointer-events: none; cursor: default; text-decoration: none;}
music_dd_data = SpacedPPs("About Music",("Preferences", "⌘")) + [UkHSplit()] + \
                 SpacedPPs(("Hide Music", "⌘H"),("Hide Others", "⇧⌘H"),("Quit Music", "⌘Q"))

# disabled = Burn Playlist to Disc | Print
file_dd_data = SpacedPPs("New" ,("Open Stream URL", "⌘U"),("Close Window", "⌘W")) + [UkHSplit()] + \
               SpacedPPs("Library",("Import", "⌘O"),("Burn Playlist to Disc", )) + [UkHSplit()] + \
               SpacedPPs(("Show in Finder", "⇧⌘R"),("Convert", )) + [UkHSplit()] + \
               SpacedPPs(("Page Setup", ),"Print")

# disabled = Undo, Redo, Cut, Copy, Paste, Deselect All
edit_dd_data = [("Undo", "⌘Z"),("Redo", "⇧⌘Z"),UkHSplit(),("Cut", "⌘X"),("Copy", "⌘C"),("Paste", "⌘V"),
                UkHSplit(),("Select All", "⌘A"),("Deselect All", "⇧⌘A"),UkHSplit(),
                SpacedTxtIcon('Smart Dictation','microphone',0.5), SpacedTxtIcon("Emojis & Symbols",'world',0.5)]

#disabled = Show Status Bar | Enter Full Screen
view_dd_data = [("Show Playing Next", ""),("Show Lyrics", ""),("Show Status Bar", ""),("Hide Sidebar", ""),("Enter Full Screen", "")]

# header: Switch account
account_dd_data = [("Switch Account", ""),UkHSplit(),("Andy", ""),SpacedTxtIcon("Benoit", 'plus-circle',0.5),("Luis", ""),UkHSplit(),
                   ("Manage Family", ""),UkHSplit(),("Add Account", "")]

In [147]:
#| export
def UkNavbarDropdown(*c,label):
    return Li(A(label,href='#'),
        Div(cls='uk-navbar-dropdown')(
            Ul(cls='uk-nav uk-navbar-dropdown-nav')(*map(Li,c))))

In [95]:
#| export
def UkNavbar(*c, cls=''):
    res = Div(cls='uk-navbar container '+ cls, uk_navbar=True)(
        Div(cls='uk-navbar-left')(
            Ul(cls='uk-navbar-nav')(*c)))
    return res

In [96]:
#|export
music_headers = UkNavbar(
      UkNavbarDropdown(*map(lambda x: A(x,href='#'),music_dd_data),label='Music'),
      UkNavbarDropdown(*map(lambda x: A(x,href='#'),file_dd_data),label='File'),
      UkNavbarDropdown(*map(lambda x: A(x,href='#'),edit_dd_data),label='Edit'),
      UkNavbarDropdown(*map(lambda x: A(x,href='#'),view_dd_data),label='View'),
      UkNavbarDropdown(*map(lambda x: A(x,href='#'),account_dd_data),label='Account'))

In [11]:
#| export
def page():
    return Div(
        music_headers,
        Div(cls="grid grid-cols-5")(
            Div(cls="hidden space-y-4 py-4 lg:block"),
            Div(cls="col-span-5 border-l border-border lg:col-span-4")))

In [12]:
show(page())

In [13]:
%%aip 0

Implement Discover, Library, Playlists left sidebar

make sure the HTML matches $`music`

In [14]:
#| export
def create_sidebar_section(title, items):
    return Div(cls="px-3 py-2")(
        H2(title, cls="mb-2 px-3 text-lg font-semibold tracking-tight"),
        Ul(cls="uk-nav uk-nav-secondary")(
            *[Li(cls="uk-active" if i == 0 and title == "Discover" else None)(
                A(cls="font-medium", href="#demo", uk_toggle=True)(
                    Span(cls="mr-2 size-4")(Span(uk_icon=f"icon: {item['icon']}; ratio: 0.8")),
                    item['text']
                )
            ) for i, item in enumerate(items)]
        )
    )

def create_playlists_section(title, items):
    return Div(cls="px-3 py-2")(
        H2(title, cls="mb-2 px-3 text-lg font-semibold tracking-tight"),
        Div(cls="max-h-60 overflow-y-auto")(
            Ul(cls="uk-nav uk-nav-secondary")(
                *[Li(
                    A(cls="font-medium", href="#demo", uk_toggle=True)(
                        Span(cls="mr-2 size-4")(Span(uk_icon="icon: play; ratio: 0.9")),
                        item
                    )
                ) for item in items]
            )
        )
    )

def left_sidebar():
    discover_items = [
        {"text": "Listen Now", "icon": "play-circle"},
        {"text": "Browse", "icon": "thumbnails"},
        {"text": "Radio", "icon": "rss"},
    ]
    
    library_items = [
        {"text": "Playlists", "icon": "play-circle"},
        {"text": "Songs", "icon": "bell"},
        {"text": "Made for You", "icon": "user"},
        {"text": "Artists", "icon": "users"},
        {"text": "Albums", "icon": "bookmark"}
    ]
    
    playlist_items = [
        "Recently Added", "Recently Played", "Top Songs",
        "Top Albums", "Top Artists", "Logic Discography",
        "Bedtime Beats", "I miss Y2K Pop"
    ]
    
    return Div(cls="space-y-4 py-4 lg:block")(
        create_sidebar_section("Discover", discover_items),
        create_sidebar_section("Library", library_items),
        create_playlists_section("Playlists", playlist_items)
    )

In [15]:
%%aip 0

remove `create_playlists_section`. instead have the create_sidebar_section function (which you should rename to something shorter by the way) accept a max_num_pre_scroll integer and decide whether to add a scrollable div based on number of items

In [16]:
#| export

def sidebar_section(title, items, max_num_pre_scroll=6):
    content = Ul(cls="uk-nav uk-nav-secondary")(
        *[Li(cls="uk-active" if i == 0 and title == "Discover" else None)(
            A(cls="font-medium", href="#demo", uk_toggle=True)(
                Span(cls="mr-2 size-4")(Span(uk_icon=f"icon: {item['icon'] if isinstance(item, dict) else 'play'}; ratio: 0.8")),
                item['text'] if isinstance(item, dict) else item
            )
        ) for i, item in enumerate(items)]
    )
    
    if max_num_pre_scroll and len(items) > max_num_pre_scroll:
        content = Div(cls="max-h-60 overflow-y-auto")(content)
    
    return Div(cls="px-3 py-2")(
        H2(title, cls="mb-2 px-3 text-lg font-semibold tracking-tight"),
        content
    )

def left_sidebar():
    discover_items = [
        {"text": "Listen Now", "icon": "play-circle"},
        {"text": "Browse", "icon": "thumbnails"},
        {"text": "Radio", "icon": "rss"},
    ]
    
    library_items = [
        {"text": "Playlists", "icon": "play-circle"},
        {"text": "Songs", "icon": "bell"},
        {"text": "Made for You", "icon": "user"},
        {"text": "Artists", "icon": "users"},
        {"text": "Albums", "icon": "bookmark"}
    ]
    
    playlist_items = [
        "Recently Added", "Recently Played", "Top Songs",
        "Top Albums", "Top Artists", "Logic Discography",
        "Bedtime Beats", "I miss Y2K Pop"
    ]
    
    return Div(cls="space-y-4 py-4 lg:block")(
        sidebar_section("Discover", discover_items),
        sidebar_section("Library", library_items),
        sidebar_section("Playlists", playlist_items, max_num_pre_scroll=5)
    )

In [17]:
%%aip 0

turn this into a FrankenSidebar class which is well documented and has a `from_json` classmethod which takes in a list indicating each sidebar section

In [18]:
#| export
from dataclasses import dataclass
from typing import List, Union, Optional

@dataclass
class SidebarItem:
    text: str
    icon: Optional[str] = None

@dataclass
class SidebarSection:
    title: str
    items: List[SidebarItem]
    max_num_pre_scroll: Optional[int] = None


class FrankenSidebar:
    """
    A class to create a sidebar with multiple sections.

    Attributes:
        sections (List[SidebarSection]): List of sidebar sections.

    Methods:
        add_section: Adds a new section to the sidebar.
        build: Builds the HTML for the entire sidebar.
        from_json: Creates a FrankenSidebar instance from a JSON-like structure.
    """

    def __init__(self):
        self.sections = []

    def add_section(self, title: str, items: List[Union[str, dict]], max_num_pre_scroll: Optional[int] = None):
        """
        Adds a new section to the sidebar.

        Args:
            title (str): The title of the section.
            items (List[Union[str, dict]]): List of items in the section. Each item can be a string or a dict with 'text' and 'icon' keys.
            max_num_pre_scroll (Optional[int]): Maximum number of items to show before adding a scroll. If None, all items are shown.
        """
        sidebar_items = [SidebarItem(item['text'], item['icon']) if isinstance(item, dict) else SidebarItem(item) for item in items]
        self.sections.append(SidebarSection(title, sidebar_items, max_num_pre_scroll))

    def build(self):
        """
        Builds the HTML for the entire sidebar.

        Returns:
            FT: The FastHTML object representing the sidebar.
        """
        def build_section(section):
            content = Ul(cls="uk-nav uk-nav-secondary")(
                *[Li(cls="uk-active" if i == 0 and section.title == "Discover" else None)(
                    A(cls="font-medium", href="#demo", uk_toggle=True)(
                        Span(cls="mr-2 size-4")(Span(uk_icon=f"icon: {item.icon or 'play'}; ratio: 0.8")),
                        item.text
                    )
                ) for i, item in enumerate(section.items)]
            )
            
            if section.max_num_pre_scroll and len(section.items) > section.max_num_pre_scroll:
                content = Div(cls="max-h-60 overflow-y-auto")(content)
            
            return Div(cls="px-3 py-2")(
                H2(section.title, cls="mb-2 px-3 text-lg font-semibold tracking-tight"),
                content
            )

        return Div(cls="space-y-4 py-4 lg:block")(
            *[build_section(section) for section in self.sections]
        )

    @classmethod
    def from_json(cls, json_data: List[dict]):
        """
        Creates a FrankenSidebar instance from a JSON-like structure.

        Args:
            json_data (List[dict]): A list of dictionaries, each representing a sidebar section.
                Each dictionary should have 'title', 'items', and optionally 'max_num_pre_scroll' keys.

        Returns:
            FrankenSidebar: An instance of FrankenSidebar with the sections from the JSON data.
        """
        sidebar = cls()
        for section in json_data:
            sidebar.add_section(section['title'], section['items'], section.get('max_num_pre_scroll'))
        return sidebar.build()

In [19]:
#| export

sidebar_data = [
    {
        'title': 'Discover',
        'items': [
            {"text": "Listen Now", "icon": "play-circle"},
            {"text": "Browse", "icon": "thumbnails"},
            {"text": "Radio", "icon": "rss"},
        ]
    },
    {
        'title': 'Library',
        'items': [
            {"text": "Playlists", "icon": "play-circle"},
            {"text": "Songs", "icon": "bell"},
            {"text": "Made for You", "icon": "user"},
            {"text": "Artists", "icon": "users"},
            {"text": "Albums", "icon": "bookmark"}
        ]
    },
    {
        'title': 'Playlists',
        'items': [
            "Recently Added", "Recently Played", "Top Songs",
            "Top Albums", "Top Artists", "Logic Discography",
            "Bedtime Beats", "I miss Y2K Pop"
        ],
        'max_num_pre_scroll': 5
    }
]


In [20]:
%%aip 0

in the same way built classes for Sidebar and Navbar, can we make a flexible modular class `FrankenTabs` that can support this

```
            <ul
              class="uk-tab-alt"
              uk-switcher="connect: #component-nav; animation: uk-animation-fade"
            >
              <li class="uk-active">
                <a href="#">Music</a>
              </li>
              <li>
                <a href="#">Podcasts</a>
              </li>
              <li class="uk-disabled">
                <a class="opacity-50">Live</a>
              </li>
            </ul>
```

In [21]:
#| export
from dataclasses import dataclass, field
from typing import List, Optional, Union

@dataclass
class TabItem:
    text: str
    href: str = "#"
    active: bool = False
    disabled: bool = False
    tab_body: Union[str, FT] = field(default_factory=lambda: P("Placeholder content"))

class FrankenTabs:
    """
    A class to create a flexible and modular tab component with associated content.

    Attributes:
        items (List[TabItem]): List of tab items.
        connect_id (str): ID of the content to be connected with the tabs.
        animation (str): Animation type for tab switching.

    Methods:
        add_tab: Adds a new tab to the component.
        build: Builds the HTML for the entire tab component including content.
        from_json: Creates a FrankenTabs instance from a JSON-like structure.
    """

    def __init__(self, connect_id: str = "component-nav", animation: str = "uk-animation-fade"):
        self.items = []
        self.connect_id = connect_id
        self.animation = animation

    def add_tab(self, text: str, href: str = "#", active: bool = False, disabled: bool = False, tab_body: Union[str, FT] = None):
        """
        Adds a new tab to the component.

        Args:
            text (str): The text to display on the tab.
            href (str): The href attribute for the tab link.
            active (bool): Whether the tab should be active by default.
            disabled (bool): Whether the tab should be disabled.
            tab_body (Union[str, FT]): The content for the tab body.
        """
        self.items.append(TabItem(text, href, active, disabled, tab_body or P(f"Content for {text}")))

    def build(self):
        """
        Builds the HTML for the entire tab component including content.

        Returns:
            FT: The FastHTML object representing the tab component and its content.
        """
        def get_cls(item):
            cls = ("uk-active" if item.active else "") + (" uk-disabled" if item.disabled else "")
            return cls if len(cls) > 0 else None
        
        tabs = Ul(cls="uk-tab-alt", uk_switcher=f"connect: #{self.connect_id}; animation: {self.animation}")(
            *[Li(cls=get_cls(item))(
                A(item.text, href=item.href, cls="opacity-50" if item.disabled else None)
            ) for item in self.items]
        )

        content = Ul(id=self.connect_id, cls="uk-switcher mt-6")(
            *[Li(Div(cls="space-y-1")(item.tab_body)) for item in self.items]
        )

        return tabs, content

    @classmethod
    def from_json(cls, json_data: List[dict], connect_id: str = "component-nav", animation: str = "uk-animation-fade"):
        """
        Creates a FrankenTabs instance from a JSON-like structure.

        Args:
            json_data (List[dict]): A list of dictionaries, each representing a tab item.
                Each dictionary should have 'text', and optionally 'href', 'active', 'disabled', and 'tab_body' keys.
            connect_id (str): ID of the content to be connected with the tabs.
            animation (str): Animation type for tab switching.

        Returns:
            FrankenTabs: An instance of FrankenTabs with the tabs and content from the JSON data.
        """
        tabs = cls(connect_id, animation)
        for item in json_data:
            tabs.add_tab(
                text=item['text'],
                href=item.get('href', '#'),
                active=item.get('active', False),
                disabled=item.get('disabled', False),
                tab_body=item.get('tab_body')
            )
        return tabs.build()

In [22]:
%%aip 0

replace `Music Content` with actual HTML from $`music`

In [23]:
#| export

cover = os.path.expanduser("~/fh-frankenui-examples/images/album.svg")

default_svg = "data:image/svg+xml;base64,PHN2ZyB4bWxucz0iaHR0cDovL3d3dy53My5vcmcvMjAwMC9zdmciIHZpZXdCb3g9IjAgMCAzMDAgMzAwIj4KICA8cmVjdCB3aWR0aD0iMzAwIiBoZWlnaHQ9IjMwMCIgZmlsbD0iI2UwZTBlMCIvPgogIDxjaXJjbGUgY3g9IjE1MCIgY3k9IjE1MCIgcj0iMTAwIiBmaWxsPSIjZmZmZmZmIiBzdHJva2U9IiMwMDAwMDAiIHN0cm9rZS13aWR0aD0iOCIvPgogIDxjaXJjbGUgY3g9IjE1MCIgY3k9IjE1MCIgcj0iMzUiIGZpbGw9IiMwMDAwMDAiLz4KICA8cGF0aCBkPSJNMTUwIDMwIEwxNTAgMjcwIE0zMCAxNTAgTDI3MCAxNTAiIHN0cm9rZT0iIzAwMDAwMCIgc3Ryb2tlLXdpZHRoPSI0Ii8+CiAgPHRleHQgeD0iMTUwIiB5PSIyOTAiIGZvbnQtZmFtaWx5PSJBcmlhbCwgc2Fucy1zZXJpZiIgZm9udC1zaXplPSIyNCIgdGV4dC1hbmNob3I9Im1pZGRsZSIgZmlsbD0iIzAwMDAwMCI+QUxCVU08L3RleHQ+Cjwvc3ZnPg=="


def create_album_grid(albums, cols=4):
    return Div(cls=f"grid grid-cols-{cols} gap-x-4")(
        *[Div(cls="space-y-3")(
            Div(cls="overflow-hidden rounded-md")(
                Img(cls="transition-transform duration-200 hover:scale-105", src=album.get('cover', default_svg), alt="")
            ),
            Div(cls="space-y-1 text-sm")(
                H3(album['title'], cls="font-medium leading-none"),
                P(album['artist'], cls="text-xs text-muted-foreground")
            )
        ) for album in albums]
    )

def music_content():
    listen_now_albums = [
        {"title": "Roar", "artist": "Catty Perry"},
        {"title": "Feline on a Prayer", "artist": "Cat Jovi"},
        {"title": "Fur Elise", "artist": "Ludwig van Beethovpurr"},
        {"title": "Purrple Rain", "artist": "Prince's Cat"},
    ]
    
    made_for_you_albums = [
        {"title": "Like a Feline", "artist": "Catdonna"},
        {"title": "Livin' La Vida Purrda", "artist": "Ricky Catin"},
        {"title": "Meow Meow Rocket", "artist": "Elton Cat"},
        {"title": "Rolling in the Purr", "artist": "Catdelle",},
        {"title": "Purrs of Silence", "artist": "Cat Garfunkel"},
        {"title": "Meow Me Maybe", "artist": "Carly Rae Purrsen"},
    ]
    
    return Div(
        Div(cls="space-y-1")(
            H2("Listen Now", cls="text-2xl font-semibold tracking-tight"),
            P("Top picks for you. Updated daily.", cls="text-sm text-muted-foreground")
        ),
        Div(cls="my-4 h-[1px] w-full bg-border"),
        create_album_grid(listen_now_albums, cols=4),
        Div(cls="mt-6 space-y-1")(
            H2("Made for You", cls="text-2xl font-semibold tracking-tight"),
            P("Your personal playlists. Updated daily.", cls="text-sm text-muted-foreground")
        ),
        Div(cls="my-4 h-[1px] w-full bg-border"),
        create_album_grid(made_for_you_albums, cols=6)
    )


In [24]:
%%aip 0

add the podcast body

In [25]:
#| export

def podcast_content():
    return Div(
        Div(cls="space-y-1")(
            H2("New Episodes", cls="text-2xl font-semibold tracking-tight"),
            P("Your favorite podcasts. Updated daily.", cls="text-sm text-muted-foreground")
        ),
        Div(cls="my-4 h-[1px] w-full bg-border"),
        Div(cls="uk-placeholder flex h-[450px] items-center justify-center rounded-md")(
            Div(cls="mx-auto flex max-w-[420px] flex-col items-center justify-center text-center")(
                Span(uk_icon="icon: podcast; ratio: 2"),
                H3("No episodes added", cls="mt-4 text-lg font-semibold"),
                P("You have not added any podcasts. Add one below.", cls="mb-4 mt-2 text-sm text-muted-foreground"),
                UkButton("Add Podcast", cls=UkButtonT.primary, uk_toggle="#demo")
            )
        )
    )

In [26]:
#| export

tabs_data = [
    {"text": "Music", "active": True, "tab_body": music_content()},
    {"text": "Podcasts", "tab_body": podcast_content()},
    {"text": "Live", "disabled": True, "tab_body": P("Live content")}
]

In [27]:
#| export
def page():
    tabs, content = FrankenTabs.from_json(tabs_data)
    return Div(
        music_headers,
        Div(cls="grid grid-cols-5")(
            Div(cls="hidden space-y-4 py-4 lg:block")(
                FrankenSidebar.from_json(sidebar_data)
            ),
            Div(cls="col-span-5 border-l border-border lg:col-span-4")(
                Div(cls="px-8 py-6")(
                    Div(cls="flex items-center justify-between")(
                        Div(cls="max-w-80")(tabs),
                        Div(
                            UkButton(cls=UkButtonT.primary)(
                                Span(cls="mr-2 size-4")(
                                    Span(uk_icon="icon: plus-circle; ratio: 0.8")
                                ),
                                "Add music"
                            )
                        )
                    ),
                    content
                )
            )
        )
    )

In [41]:
#| export

music_homepage = page()

NameError: name 'page' is not defined

In [66]:
#| hide
import nbdev; nbdev.nbdev_export()